In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pypsa

In [ ]:
run = "nresults_full_3H_ws"
prefix = "/mnt/c/Users/scl38887/Documents/git/aldehyde/workflow/subworkflows/pypsa-earth-sec/results/" + run + "/postnetworks/"
output = "/mnt/c/Users/scl38887/Documents/git/aldehyde/results/" + run + "/graphics/rldc/"

In [ ]:
path = "elec_s_10_ec_lc1.0_Co2L0.50_3H_2030_0.15_DF_20export.nc"
n = pypsa.Network(prefix + path)

INFO:pypsa.io:Imported network elec_s_10_ec_lc1.0_Co2L0.50_3H_2030_0.15_DF_20export.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores


In [ ]:
#n.links[n.links.carrier == "H2 Electrolysis"].marginal_cost

In [ ]:
capex = n.links[n.links.carrier == "H2 Electrolysis"].p_nom_opt[0] * n.links[n.links.carrier == "H2 Electrolysis"].capital_cost[0]
opex = n.links_t.p0.loc[:, n.links.carrier == "H2 Electrolysis"].sum()[0] * n.links[n.links.carrier == "H2 Electrolysis"].marginal_cost[0]
el_cost = (n.links_t.p0.loc[:, n.links.carrier == "H2 Electrolysis"].iloc[:, 0] * n.buses_t.marginal_price.loc[:, n.buses.carrier == "AC"].iloc[:, 0]).sum()
hydrogen = n.links_t.p0.loc[:, n.links.carrier == "H2 Electrolysis"].iloc[:, 0].sum()

#opex_link = series_to_df(n.links_t.p0.sum() * n.links.marginal_cost)
#series_to_df(n.links.p_nom_opt * n.links.capital_cost)

In [ ]:
lcoh = (capex + opex + el_cost) / hydrogen 
lcoh

124.47962309258212

In [ ]:

n.buses_t.marginal_price.loc[:, n.buses.carrier == "H2"].iloc[:, 0].describe() #).sum()

count    2920.000000
mean      120.822785
std        60.140289
min         0.061697
25%       128.015851
50%       154.951541
75%       154.953311
max       191.550787
Name: H2 export bus, dtype: float64